In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys
import numpy as np
sys.path.append('/Users/juliankimura/Desktop/deepomics')
import deepomics.neuralnetwork as nn
from deepomics import learn, utils
from models import standard_model, all_conv_model

sys.path.append('/Users/juliankimura/Desktop/data')
import load_data

np.random.seed(247) # for reproducibility

In [2]:
data_path = '/Users/juliankimura/Desktop/data/GenomeSimulation'
file_path = os.path.join(data_path, 'Localized_N=100000_S=200_M=50_G=20_data.pickle')
train, valid, test = load_data.simulation_pickle(file_path)

loading data from: /Users/juliankimura/Desktop/data/GenomeSimulation/Localized_N=100000_S=200_M=50_G=20_data.pickle
loading train data
loading cross-validation data
loading test data


In [3]:
# build network
shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]
network, placeholders, optimization = standard_model.model(shape, num_labels)

In [4]:
# build neural network class
nnmodel = nn.NeuralNet(network, placeholders)
nnmodel.inspect_layers()

# set output file paths
output_name = 'test'
utils.make_directory(data_path, 'Results')
file_path = os.path.join(data_path, 'Results', output_name)
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', file_path=file_path)

----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: 
shape:(None, 4, 200, 1)
layer2: 
shape:(None, 30, 182, 1)
parameters: W
layer3: 
shape:(None, 30, 182, 1)
parameters: beta, gamma, mean, inv_std
layer4: 
shape:(None, 30, 182, 1)
layer5: 
shape:(None, 30, 182, 1)
layer6: 
shape:(None, 30, 9, 1)
layer7: 
shape:(None, 512)
parameters: W
layer8: 
shape:(None, 512)
parameters: beta, gamma, mean, inv_std
layer9: 
shape:(None, 512)
layer10: 
shape:(None, 512)
layer11: 
shape:(None, 20)
parameters: W
layer12: 
shape:(None, 20)
parameters: b
layer13: 
shape:(None, 20)
----------------------------------------------------------------------------
compiling model


In [5]:
# train model
learn.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=100, num_epochs=500, patience=10, verbose=1)

Epoch 1 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.12455 -- accuracy=95.91%  
  valid loss:		0.06652
  valid accuracy:	0.97591+/-0.01654
  valid auc-roc:	0.97257+/-0.03183
  valid auc-pr:		0.75724+/-0.27473
saving model parameters to: /Users/juliankimura/Desktop/data/GenomeSimulation/Results/test_best.pickle
Epoch 2 out of 500 
[====                          ] 11.9% -- time=42s -- loss=0.07147 -- accuracy=97.49%  

KeyboardInterrupt: 

In [ ]:
# load best model --> lowest cross-validation error
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(test, name="test", batch_size=100)

In [ ]:
# save all performance metrics (train, valid, test)
nntrainer.save_all_metrics(file_path)